In [1]:
from data_gen import *
from models import *
from keras import models

Using TensorFlow backend.


In [2]:
# data generator
dg = DataGen(data="data/wolayitta_clean.txt")

# length of a word
n_input_length = len(char2int)
n_steps_in = dg.max_root_len
n_steps_out = dg.max_output_len
print(n_steps_in, n_steps_out)

15 31


In [3]:
print("Total train data: ", len(dg.words) * .7)
batch_size = 128
# number of batches to train
n_batches = int(len(dg.words) * .7 / batch_size) 

# python generator to generate training data at each request
# E.x word_matrix, feature = next(gen)
gen = dg.onehot_gen_data(batch_size=batch_size, n_batches=n_batches)

Total train data:  579040.0


In [4]:
train, infenc, infdec = onehot_model(len(dg.root2int), n_input_length, dg.word_feat_len, 512)

In [5]:
train.compile(optimizer='adam', loss='categorical_crossentropy')

In [6]:
train.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
root_word_input (InputLayer)    (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 507)       1222884     root_word_input[0][0]            
__________________________________________________________________________________________________
word_feature_input (InputLayer) (None, 32)           0                                            
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 507)          0           embedding_1[0][0]                
__________________________________________________________________________________________________
feature_ou

In [7]:
history = train.fit_generator(gen, steps_per_epoch=n_batches, epochs = 3)

Epoch 1/3
4523/4523 [==============================] - 874s 193ms/step - loss: 0.1199
Epoch 2/3
4523/4523 [==============================] - 878s 194ms/step - loss: 0.0036
Epoch 3/3
 581/4523 [==>...........................] - ETA: 12:44 - loss: 0.0027

KeyboardInterrupt: 

In [ ]:
# test_n_batches, test_batch_size =  int(len(dg.words) * .7 / batch_size), batch_size  
test_n_batches, test_batch_size = 30, 100 

# data generator for test data
test_gen = dg.onehot_gen_data(batch_size=test_batch_size, n_batches=test_n_batches, trainset=False)

In [ ]:
# shows sample examples and calculates accuracy

total, correct = 0, 0
in_word = 0
sims = []
for b in range(test_n_batches):
    # get data from test data generator
    [X1, X2, X3], y = next(test_gen)
    for j in range(test_batch_size):
        word_features = X3[j].reshape((1, X3.shape[1])) 
        root_word_matrix = X1[j].reshape((1, 1))
        
        # predicts the target word given root word and features
        target = predict(infenc, infdec, root_word_matrix, word_features, n_steps_out, n_input_length)
        root = dg.int2word[X1[j]]#.replace('&', ' ')
        word = ''.join(dg.one_hot_decode(y[j]))#.replace('&', ' ')
        targetS = ''.join(dg.one_hot_decode(target))#.replace('&', ' ')
        sims.append(dg.word_sim(word, targetS))
#         print(root)
        # checks if the predicted and the real words are equal
        if dg.one_hot_decode(y[j]) == dg.one_hot_decode(target):
            correct += 1
#         else:
#             print(root, word.split('&')[0], '\t\t', targetS.split('&')[0])
#         if root.strip() in targetS.strip():
#             in_word += 1
#     print(b, root, word, targetS)
    total += test_batch_size
    

print('Exact Accuracy: %.2f%%' % (float(correct)/float(total)*100.0))

In [ ]:
int2word = {dg.int2word[key]: key for key in list(dg.int2word)}

In [ ]:
[256, 28.47, 35.45, 35.76, 128, 4.97, 18.20]